In [1]:
import pandas as pd
import numpy as np
import timeit
from scipy.optimize import curve_fit

In [2]:
df_raw_2012 = pd.read_excel("../data/processed/bronze/2012_Charge-7°C.xlsx", index_col=False)
df_raw_2012.head()

,Unnamed: 0,t[s],V[mph],F[N],T[°C],SOC[%],U[V],I[A],t_acumulated[s],V[m/s],a[m/s2]
0,0,-10.0,-0.004166,-2.738338,-5.880604,9.674727,0.000000,0.0,0.0,-0.001862,0.000000
1,1,-9.9,-0.004166,-2.738338,-5.879956,9.675699,12.027976,0.0,0.1,-0.001862,0.000000
2,2,-9.8,-0.004490,-2.738338,-5.881252,9.673754,12.024676,0.0,0.2,-0.002007,-0.001448
3,3,-9.7,-0.004166,-2.738338,-5.879632,9.675375,12.021376,0.0,0.3,-0.001862,0.001448
4,4,-9.6,-0.004490,-2.738338,-5.881576,9.675375,12.018077,0.0,0.4,-0.002007,-0.001448


In [3]:
df_raw_2013 = pd.read_excel("../data/processed/bronze/2013_Charge 23°C.xlsx")
df_raw_2013.head()

,Unnamed: 0,t[s],V[mph],F[N],T[°C],SOC[%],U[V],I[A],t_acumulated[s],V[m/s],a[m/s2]
0,0,-10.0,0.009,-12.170,22.256,0.0,-0.043,0.438,0.0,0.004023,0.00000
1,1,-9.9,0.010,-12.332,22.256,0.0,-0.037,0.437,0.1,0.004470,0.00447
2,2,-9.8,0.010,-12.272,22.256,0.0,-0.048,0.435,0.2,0.004470,0.00000
3,3,-9.7,0.009,-12.146,22.256,0.0,-0.052,0.438,0.3,0.004023,-0.00447
4,4,-9.6,0.010,-12.240,22.255,0.0,-0.050,0.439,0.4,0.004470,0.00447


# Pre-calculated parameters


$ F = M∙ (k ∙ a + μ ⋅ g)+\dfrac{C_D⋅ A_f ⋅ ρ_{air} ⋅V^2}{2}$

In [4]:
def calculate_force(df):
    μ = 0
    k=1.15
    ρ_air = 1.2
    g = 10
    M = 1285
    C_d = 0.2475
    A_f =  2.27
    accelation = df['a[m/s2]']
    velocity = df['V[m/s]']
    df['F_calc[N]'] = M*(k*accelation + μ*g) + C_d*A_f*ρ_air*(velocity**2)/2
    return df

In [5]:
def calculate_force_v2(df, coeffs):
    M = coeffs[0]
    C_d = coeffs[1]
    μ = coeffs[2]
    k = coeffs[3]
    ρ_air = coeffs[4]
    g = coeffs[5]
    A_f =  coeffs[6]
    accelation = df['a[m/s2]']
    velocity = df['V[m/s]']
    df['F_calc[N]'] = M*(k*accelation + μ*g) + C_d*A_f*ρ_air*(velocity**2)/2
    return df['F_calc[N]']

$ i_{F>0} = i_0 +  \dfrac{V ⋅F}{U⋅ 𝜀_{drive}} $

$ i_{F<0} = i_0 + \dfrac{V ⋅F}{U⋅ 𝜀_{regen}}
$ 

In [6]:
def calculate_current(df):
    i_0 = 8.685
    E_drive = 0.8314
    E_regen = 1.048
    velocity = df['V[m/s]']
    force = df['F_calc[N]']
    voltage = df['U[V]']
    
    df.loc[df['F_calc[N]'] > 0, 'I_calc[A]'] = i_0 + (velocity*force)/(voltage*E_drive)
    df.loc[df['F_calc[N]'] < 0, 'I_calc[A]'] = i_0 + (velocity*force)/(voltage*E_regen)
    return df

In [7]:
def calculate_current_v2(df, coeffs): 
    i_0 = coeffs[0]
    E_drive = coeffs[1]
    E_regen = coeff[2]
    velocity = df['V[m/s]']
    force = df['F_calc[N]']
    voltage = df['U[V]']
    
    df.loc[df['F_calc[N]'] > 0, 'I_calc[A]'] = i_0 + (velocity*force)/(voltage*E_drive)
    df.loc[df['F_calc[N]'] < 0, 'I_calc[A]'] = i_0 + (velocity*force)/(voltage*E_regen)
    return df

$T = \dfrac{τ}{η} + (T_{ini} −\dfrac{τ}{η}) ⋅ e^{-η⋅t} $

$ 𝜏 = \dfrac{R_{ohmic}⋅ 𝑖^2 + C_{entrop}⋅ 273 ⋅ 𝑖 +UA_{𝑝𝑎𝑐𝑘}⋅ T_{𝑎𝑚𝑏} − Q_0}{C_{𝑡ℎ𝑒𝑟𝑚}}$

$ 𝜂 = \dfrac{UA_{𝑝𝑎𝑐𝑘} − C_{𝑒𝑛𝑡𝑟𝑜𝑝} ∙ 𝑖}{C_{therm}}$

In [8]:
def calculate_temperature(df):
    UA_pack = 70
    R_ohmic = 0
    C_entrop = -73
    T_amb = 22 + 273
    Q_0 = 8923
    C_therm = 200000
    T_ini = 1
    

    df.loc[df.index[0], 'T_calc[°C]'] = 0
    # Calculate the SOC column iteratively
    for i in range(1, len(df)):
        current = df.loc[df.index[i], 'I_calc[A]']
        𝜏 = (R_ohmic*current**2 + 273*C_entrop*current + UA_pack*T_amb - Q_0) / C_therm
        𝜂 = (UA_pack - C_entrop*current)/C_therm
        past_temp = df.loc[df.index[i-1], 'T_calc[°C]']
        time = df.loc[df.index[i-1], 't[s]']
        df.loc[df.index[i], 'T_calc[°C]'] = 𝜏/𝜂 + (past_temp - 𝜏/𝜂) * np.exp(-𝜂*time)
    return df

In [9]:
def calculate_temperature_v2(df, coeffs): 
    UA_pack = coeffs[0]
    R_ohmic = coeffs[1]
    C_entrop = coeffs[2]
    T_amb = coeffs[4]
    Q_0 = coeffs[5]
    C_therm = coeffs[6]
    T_ini = coeffs[7]
    
    df.loc[df.index[0], 'T_calc[°C]'] = 0
    # Calculate the SOC column iteratively
    for i in range(1, len(df)):
        current = df.loc[df.index[i], 'I_calc[A]']
        𝜏 = (R_ohmic*current**2 + 273*C_entrop*current + UA_pack*T_amb - Q_0) / C_therm
        𝜂 = (UA_pack - C_entrop*current)/C_therm
        past_temp = df.loc[df.index[i-1], 'T_calc[°C]']
        time = df.loc[df.index[i-1], 't[s]']
        df.loc[df.index[i], 'T_calc[°C]'] = 𝜏/𝜂 + (past_temp - 𝜏/𝜂) * np.exp(-𝜂*time)
    return df

In [10]:
df_filtered_2013 = (df_raw_2013.pipe(calculate_force)
                                    .pipe(calculate_current)
                                    .pipe(calculate_temperature))
df_filtered_2013.head()

C:\Users\muril\AppData\Local\Temp\ipykernel_12164\155084255.py:19: RuntimeWarning: overflow encountered in double_scalars
  df.loc[df.index[i], 'T_calc[°C]'] = 𝜏/𝜂 + (past_temp - 𝜏/𝜂) * np.exp(-𝜂*time)
C:\Users\muril\AppData\Local\Temp\ipykernel_12164\155084255.py:19: RuntimeWarning: overflow encountered in exp
  df.loc[df.index[i], 'T_calc[°C]'] = 𝜏/𝜂 + (past_temp - 𝜏/𝜂) * np.exp(-𝜂*time)
C:\Users\muril\AppData\Local\Temp\ipykernel_12164\155084255.py:19: RuntimeWarning: invalid value encountered in double_scalars
  df.loc[df.index[i], 'T_calc[°C]'] = 𝜏/𝜂 + (past_temp - 𝜏/𝜂) * np.exp(-𝜂*time)
C:\Users\muril\AppData\Local\Temp\ipykernel_12164\155084255.py:15: RuntimeWarning: invalid value encountered in double_scalars
  𝜏 = (R_ohmic*current**2 + 273*C_entrop*current + UA_pack*T_amb - Q_0) / C_therm


,Unnamed: 0,t[s],V[mph],F[N],T[°C],SOC[%],U[V],I[A],t_acumulated[s],V[m/s],a[m/s2],F_calc[N],I_calc[A],T_calc[°C]
0,0,-10.0,0.009,-12.170,22.256,0.0,-0.043,0.438,0.0,0.004023,0.00000,0.000005,8.684999,0.000000
1,1,-9.9,0.010,-12.332,22.256,0.0,-0.037,0.437,0.1,0.004470,0.00447,6.605952,7.725030,7.225157
2,2,-9.8,0.010,-12.272,22.256,0.0,-0.048,0.435,0.2,0.004470,0.00000,0.000007,8.684999,15.609318
3,3,-9.7,0.009,-12.146,22.256,0.0,-0.052,0.438,0.3,0.004023,-0.00447,-6.605940,9.172693,24.721840
4,4,-9.6,0.010,-12.240,22.255,0.0,-0.050,0.439,0.4,0.004470,0.00447,6.605952,7.974622,32.769535


In [11]:
df_filtered_2013.describe()

C:\Users\muril\conda\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,Unnamed: 0,t[s],V[mph],F[N],T[°C],SOC[%],U[V],I[A],t_acumulated[s],V[m/s],a[m/s2],F_calc[N],I_calc[A],T_calc[°C]
count,99813.000000,99813.000000,99813.000000,99813.000000,99813.000000,99813.000000,99813.000000,99813.000000,99813.000000,99813.000000,9.981300e+04,99813.000000,9.981300e+04,3119.000000
mean,5531.798423,543.179368,32.480254,104.607169,23.379180,51.544461,19.048800,365.429255,4990.600000,14.519559,1.343594e-07,102.151566,NaN,inf
std,3698.025316,369.802118,21.481920,903.610864,2.508481,28.546150,29.450568,40.104654,2881.367555,9.603004,6.019669e-01,894.889909,NaN,NaN
min,0.000000,-10.000000,-0.239000,-4970.851000,20.190000,0.000000,-115.445000,0.432000,0.000000,-0.106840,-3.388467e+00,-5002.902250,-inf,-272.969776
25%,2495.000000,239.500000,16.350000,-12.523000,20.927000,25.200000,0.536000,359.399000,2495.300000,7.308896,-1.028163e-01,-15.449164,7.125125e+01,494.478074
50%,4990.000000,489.000000,28.858000,187.021000,22.458000,48.000000,18.588000,370.696000,4990.600000,12.900313,0.000000e+00,171.372408,2.010365e+02,NaN
75%,7972.000000,787.199000,55.009000,348.327000,25.939000,77.500000,34.444000,383.024000,7485.900000,24.590523,1.609298e-01,356.052850,3.315720e+02,NaN
max,13870.000000,1376.999000,80.642000,5873.214000,28.600000,102.300000,247.719000,396.053000,9981.200000,36.049173,3.947251e+00,5843.090926,inf,inf


# New parameters

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math

def calculate_metrics(true, predict):
    MAE = mean_absolute_error(true, predict)
    MSE = mean_squared_error(true, predict)
    RMSE = math.sqrt(MSE)
    R2 = r2_score(true, predict)
    print("MAE: ", MAE, "\nMSE: ", MSE, "\nRMSE: ", RMSE, '\nR²: ', R2)
    return MAE, MSE, RMSE, R2

In [13]:
df_opt_2013 = df_raw_2013.copy()

In [14]:
start = timeit.default_timer()

def force_opt(x, M, C_d):
    μ = 0
    k =1.15
    ρ_air = 1.2
    g = 10
    A_f = 2.27
    accelation = x['a[m/s2]']
    velocity = x['V[m/s]']
    y = M*(k*accelation + μ*g) + C_d*A_f*ρ_air*(velocity**2)/2
    return y

x= df_opt_2013
y =np.array(df_opt_2013['F[N]'])
initial_guess =  [400,5, 10, 10, 10, -10, 30, 60]
force_python_coeffs, pcov = curve_fit(force_opt, xdata = x, ydata = y,maxfev=40000)
print(force_python_coeffs)
print("Duration: ", timeit.default_timer() - start)

[1.28470081e+03 2.66449302e-01]
Duration:  0.21061629999996967


In [15]:
μ = 0
k = 1.15
ρ_air = 1.2
g = 10
A_f =  2.27

print("Excel results")
M = 1285
C_d = 0.2475
force_excel_coeffs = [M, C_d, μ, k, ρ_air, g, A_f]
force_excel_result = calculate_force_v2(df_opt_2013, force_excel_coeffs)
force_MAE_excel, force_MSE_excel, force_RMSE_excel, force_R2_excel = calculate_metrics(df_opt_2013['F[N]'], force_excel_result)


print("\nPython results")
force_python_coeffs = np.concatenate((force_python_coeffs, [μ, k, ρ_air, g, A_f]))
force_python_result = calculate_force_v2(df_opt_2013, force_python_coeffs)
force_MAE_python, force_MSE_python, force_RMSE_python, force_R2_python = calculate_metrics(df_opt_2013['F[N]'], force_python_result)

Excel results
MAE:  72.11158283342519 
MSE:  13528.025505532542 
RMSE:  116.31004043302771 
R²:  0.9834317790802255

Python results
MAE:  71.27380701737499 
MSE:  13412.748002638482 
RMSE:  115.81341892301808 
R²:  0.9835729632563085


In [16]:
start = timeit.default_timer()

def current_opt(x, i_0, E_drive, E_regen):
    velocity = x['V[m/s]']
    force = x['F_calc[N]']
    voltage = x['U[V]']
    x.loc[x['F_calc[N]'] > 0, 'I_calc[A]'] = i_0 + (velocity*force)/(voltage*E_drive)
    x.loc[x['F_calc[N]'] < 0, 'I_calc[A]'] = i_0 + (velocity*force)/(voltage*E_regen)
    y = x['I_calc[A]']
    return y

x= df_opt_2013
y = np.array(df_opt_2013['I[A]'])
current_python_coeffs, pcov = curve_fit(current_opt, xdata = x, ydata = y,maxfev=100000)
print(current_python_coeffs)
print("Duration: ", timeit.default_timer() - start)

RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 100000.

In [ ]:
print("Excel results")
i_0 = 8.685
E_drive = 0.8314
E_regen = 1.048
current_excel_coeffs = [i_0, E_drive, E_regen]
current_excel_result = calculate_current_v2(df_opt_2013, current_excel_coeffs)
current_MAE_excel, current_MSE_excel, current_RMSE_excel, current_R2_excel = calculate_metrics(df_opt_2013['I[A]'], current_excel_result)

print("\nPython results")
current_python_result = calculate_force_v2(df_opt_2013, current_python_coeffs)
current_MAE_python, current_MSE_python, current_RMSE_python, current_R2_python = calculate_metrics(df_opt_2013['I[A]'], current_python_result)

In [ ]:
start = timeit.default_timer()

def temperature_opt(x, UA_pack, R_ohmic, C_entrop):
    T_amb = 22 + 273
    Q_0 = 8923
    C_therm = 200000    
    df.loc[df.index[0], 'T_calc[°C]'] = 0
    # Calculate the SOC column iteratively
    for i in range(1, len(df)):
        current = df.loc[df.index[i], 'I_calc[A]']
        𝜏 = (R_ohmic*current**2 + 273*C_entrop*current + UA_pack*T_amb - Q_0) / C_therm
        𝜂 = (UA_pack - C_entrop*current)/C_therm
        
        past_temp = df.loc[df.index[i-1], 'T_calc[°C]']
        time = df.loc[df.index[i-1], 't[s]']
        df.loc[df.index[i], 'T_calc[°C]'] = 𝜏/𝜂 + (past_temp - 𝜏/𝜂) * np.exp(-𝜂*time)
    return df['T_calc[°C]']

x= df_opt_2013
y =np.array(df_opt_2013['T[°C]'])
initial_guess =  [400,5, 10, 10, 10, -10, 30, 60]
temperature_python_coeffs, pcov = curve_fit(temperature_opt, xdata = x, ydata = y,maxfev=1000)
print(temperature_python_coeffs)
print("Duration: ", timeit.default_timer() - start)


In [ ]:
T_amb = 22 + 273
Q_0 = 8923
C_therm = 200000
T_ini = 1

print("Excel results")
UA_pack = 70
R_ohmic = 0
C_entrop = -73
temperature_excel_coeffs = [UA_pack, R_ohmic, C_entrop, T_amb, Q_0, C_therm, g, T_ini]
temperature_excel_result = calculate_temperature_v2(df_opt_2013, temperature_excel_coeffs)
temperature_MAE_excel, temperature_MSE_excel, temperature_RMSE_excel, temperature_R2_excel = calculate_metrics(df_opt_2013['T[°C]'], temperature_excel_result)

print("\nPython results")
temperature_python_coeffs = np.concatenate((temperature_python_coeffs, [T_amb, Q_0, C_therm, g, T_ini]))
temperature_python_result = calculate_temperature_v2(df_opt_2013, temperature_python_coeffs)
temperature_MAE_python, temperature_MSE_python, temperature_RMSE_python, temperature_R2_python = calculate_metrics(df_opt_2013['T[°C]'], temperature_python_result)